# HW3, CHEE 6397 (Data-Driven Materials Modeling)

#### **Topics**: Unsupervised Learning -- Dimension Reduction & Clustering

#### **Due**: November 7, 2023


## Instructions

- The file you submit should be named as `hw3-<FirstName>_<LastName>-<UHID>.ipynb`, e.g. `hw3-Mingjian_Wen-00001111.ipynb`.
- Input your answer to each question in the `Answer` cell. Feel free add to more cells as needed.

- Do not delete the question and scores cells.
 
### Markdown and Math

- You can use `markdown` cell to type the text part of your answers to a question. And math equations can be typed using LaTex.
- See https://gtribello.github.io/mathNET/assets/notebook-writing.html for a quick intro to markdown in Jupyter and how to write math equations.
- See https://ia.wikipedia.org/wiki/Wikipedia:LaTeX_symbols for a list of LaTex symbols.
- If you are more used to MS Word equation typesetting, you can find a graphical LaTex equations generator at: https://latexeditor.lagrida.com

### Code

- Your Python code to a question can be written using the `code` cell.
- Make sure all used packages are imported properly. For example, before submission, do `Kernel->Restart and Run All Cells...` from the menu bar to double check. If we cannot run your notebook, we won't be able to grade it.


## Scores

Problem 1

- a [5 points]:
- b [15 points]:
- c [10 points]:
- d [20 points]:
- e [10 points]:

Problem 2

- a [5 points]:
- b [10 points]:
- c [5 points]:
- d [10 points]:
- e [10 points]:

Total: 


---
---


## Problem 1

In this problem, we explore the use of PCA and t-SNE/UMAP/TMAP for dimension reduction and visualization of high-dimensional data.

### Dataset 

The dataset we will use is the USPTO chemical reactions data. We are not going to use the original dataset, but a preprocessed version with the features already computed. 
The preprocessed data is from the bleow paper:

- *Improving machine learning performance on small chemical reaction data with unsupervised contrastive pretraining* by Wen et al,
- DOI: https://doi.org/10.1039/D1SC06515G

The purpose here is to visualize the reactions by their family, like what is done in Fig. 5 (fine-tuned) in the paper.



### Data files 

Two files are provided: 
- `reaction_features_finetuned.npy` consists of the features of reactions computed by a fined-tuned model detailed in the paper (for this problem, we don't need to worry about how they are computed). The data is a 2D array, and each row is the features for a reaction. 
- `reaction_super_family.npy` consists of the super family labels for each reaction (the values are 1, 2, ... 8; see Fig. 5 in the paper for what they mean). The data is a 1D array, and each element is the super family label for a reaction.
- The two files are in consistent order, e.g., the first row in `reaction_features_finetuned.npy` is the features of the first reaction, and its super family label is the first component in `reaction_super_family.npy`.

### (a)

- Load the features matrix and assign it to `X`.
- Load the labels and assign to `y`.
- How many chemical reactions (data points) are there in the dataset?
- How many features are there for each reaction?

Hint: `numpy.load()` can be used to read `.npy` files.

### Answer


### (b)

- Perform PCA for the features `X` and reduce the dimension to 2.
- Plot the first two principal components of the reactions (2D scatter plot). Color the reactions by their super family labels `y`. 


### Answer  

### (c)

- Is PCA with two principle components good enough for capturing the patterns in the reactions? Why or why not?
- If we want to obtain an explained variance ratio of 0.8, how many principal components do we need to keep?

### Answer

### (d)

- Select one of t-SNE, UMAP, or TMAP, and perform dimension reduction for the features `X` to 2D. (UMAP/TMAP is not available in scikit-learn, you need to install from their respective repos.)
- Plot the 2D embedding of the reactions. Color the reactions by their super family labels `y`.
- You might need to tune the hyperparameters of the dimension reduction method to get a good result. Explain what are the import hyperparameters you choose to tune and how you tune them. 

### Answer

### (e)

- Is the visualization obtained from t-SNE/UMAP/TMAP better than that by PCA? Why or why not?
- How does the visualization compare with Fig. 5 (Fine-tuned) in the paper? If it is not as good, any explanation why?

### Answer

---


## Problem 2

This problem evaluate the performance of k-means for the USPTO chemical reactions data used in Problem 1.


In most cases, we don't have the true labels for the data. Then the quality of a clustering algorithm can be evaluated by metrics such as the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)#:~:text=The%20silhouette%20value%20is%20a,poorly%20matched%20to%20neighboring%20clusters.), which measures how similar a data point is to its own cluster compared to other clusters. 

Here, we have the super family labels for the reactions, and we can then evaluate the performance of the clustering algorithms using classification metrics.

### (a)

- Perform k-means clustering on the features `X`. Note, set `n_clusters` parameter for k-means to get the correct number of clusters of you are using scikit-learn.
- Then obtain the cluster info for the reactions.  

### Answer 

### (b)

Compute the precision, recall, and F1 scores for the k-means clustering results. This is a multi-class problem, so if you are using the scikit-learn metric functions, you need to set the `average` parameter properly. Justify your choice of the `average` parameter, i.e. why you choose that particular value.


We can evaluate the performance of k-means by comparing the `predicted labels` with the `super family labels` (`y` in problem 1b). Here, it is a bit tricky to get the `predicted labels` -- one cannot simply use the cluster index as the predicted label, because the numbering of the cluster does not matter. For example, the indices of cluster 0 and cluster 3 can be switched and it is still the same clustering.

Therefore, we need to find a way to match each cluster to a super family label. This is achieved in the below way: for each cluster, we find the super family label that is most frequent in the cluster, and assign this label for all data points in the cluster. You can use the `convert_clusters_to_predicted_labels()` function below to achieve this.


In [2]:
import collections
import numpy as np

def convert_clusters_to_predicted_labels(clusters, true_labels):
    """
    Assign predicted label for the cluster data points.

    The predicted label is the most frequent true label in the cluster. All data points
    in a cluster are assigned the same predicted label.

    Args:
        clusters (1D array): predicted cluster info for each sample
        true_labels (1D array): true label (super family) for each sample

    Returns:
        1D array: predicted label for each sample
    """
    clusters = np.asarray(clusters)
    true_labels = np.asarray(true_labels)

    clusters_set = list(set(clusters))

    predicted_label = np.zeros(len(true_labels)) - 1
    for c in clusters_set:
        mask = (clusters == c)
        true_lb = true_labels[mask]
        pred_lb = collections.Counter(true_lb).most_common(1)[0][0]
        predicted_label[mask] = pred_lb

    if not (predicted_label> 0).all():
        raise ValueError('Some samples are not assigned to any cluster')

    return predicted_label


### Answer

### (c)

Based on your metric scores in (b), how good is k-means for capturing the super family labels? Justify your answer.

### Answer

### (d)

We suspect that the performance might be improved (i.e. higher precision, recall, and F1 score) if we first use PCA to reduce the dimensionality of the features, and then perform k-means clustering on the reduced features. Let's verify our suspicion, by doing the following:
- Perform PCA on the features `X` to reduce the dimensionality. 
- Perform k-means clustering on the reduced features and then compute the precision, recall, and F1 score for the k-means clustering results.

Hint: 
- You might need to tune the number of principal components to keep in PCA to get the best performance. 
- If you are using scikit-learn, both the `PCA` and `KMeans` classes have random states, and you can set them to make the results reproducible.

### Answer 

### (e) 
- What is the number of principal components you choose in (d) that gives the best performance? 
- Is our hypothesis in (d) correct? If yes, any reason why PCA helps? If not, any explanation why?

### Answer